In [4]:
import pandas as pd
from datetime import datetime
# import matplotlib.pyplot as plt
# import numpy as np

In [5]:
do_clean = False

In [6]:
if do_clean:
    path = ""
else:
    path = "clean/"
    
# file = "sessions_161718_top1000.csv"
# file = "sessions_161718_95014_top10.csv"
file = "sessions_161718_95014_tail10.csv"

df = pd.read_csv(path + file, index_col=False) 

print(df.head(3))


    Port ID  Energy (kWh) Station Start Time (Local)  Session Time (secs)
0  123291.0     24.346367        2016-01-01 16:41:20                14979
1  121533.0      2.449812        2016-01-04 07:36:51                 2831
2  123291.0     11.995313        2016-01-04 08:18:16                33836


In [7]:
if do_clean:
    # remove noise:
    df = df[df["Session Time (secs)"] > 5*60]
    df["Session Time (secs)"] = df["Session Time (secs)"].apply(lambda x: min(x, 24*3600))
    df = df[df["Energy (kWh)"] > 1]
    df["Energy (kWh)"] = df["Energy (kWh)"].apply(lambda x: min(x, 100))

    # convert to datetimes and sort
    df['Station Start Time (Local)'] = df['Station Start Time (Local)'].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df = df.sort_values(['Station Start Time (Local)']) 
    df = df[['Station Start Time (Local)', "Energy (kWh)", "Session Time (secs)"]]


In [8]:
if do_clean:
    df.to_csv(path + "clean/"  + file, index=False)

In [9]:
def session_csv2station_list(df):
    # Ports
    station_id = df["Port ID"].unique()
#     print(station_id)

    # convert to datetimes and sort
    df['Station Start Time (Local)'] = df['Station Start Time (Local)'].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    df = df.sort_values(['Station Start Time (Local)']) 

    # convert to hours
    df['Session Time (secs)'] = df['Session Time (secs)'] / 3600.0

    # iterate: split ports, create tuples
    station_list = []
    for this_id in station_id:
        df_id = df[df["Port ID"] == this_id]
        df_id = df_id.drop(columns=["Port ID"])
        station_list.append(list(df_id.itertuples(index=False, name=None)))  
        
    return station_list

In [10]:
station_list = session_csv2station_list(df)


In [11]:
# print(station_list_tail)
for entry in station_list[3][:10]:
    print(entry)  

for entry in station_list[3][-10:]:
    print(entry)  

(15.3313, Timestamp('2016-04-11 22:24:21'), 3.1594444444444445)
(18.414671, Timestamp('2016-04-12 09:34:06'), 3.5083333333333333)
(8.304698, Timestamp('2016-04-13 10:28:21'), 1.3747222222222222)
(7.185753, Timestamp('2016-04-13 13:41:34'), 3.7305555555555556)
(5.625832, Timestamp('2016-04-14 09:15:05'), 8.960277777777778)
(12.537595, Timestamp('2016-04-15 09:20:16'), 7.104722222222223)
(53.248348, Timestamp('2016-04-18 09:57:12'), 9.335277777777778)
(10.758947000000001, Timestamp('2016-04-19 09:11:15'), 5.488333333333333)
(12.043006, Timestamp('2016-04-20 10:46:07'), 3.4252777777777776)
(4.709206, Timestamp('2016-04-20 14:13:04'), 4.906944444444444)
(30.753384000000004, Timestamp('2018-10-26 08:34:06'), 5.106111111111111)
(3.5044839999999997, Timestamp('2018-10-26 13:52:34'), 4.276666666666666)
(6.270281, Timestamp('2018-10-29 09:36:58'), 1.9908333333333332)
(19.151569, Timestamp('2018-10-29 11:44:52'), 5.282222222222222)
(3.602533, Timestamp('2018-10-29 17:25:04'), 1.0730555555555557)